In [ ]:
#ADICIONAR PERTURBAÇÃO NAS FEATURES DA INSTANCIA ESCOLHIDA PARA VALIDAR A PI-EXPLICAÇÃO
import numpy as np
import pandas as pd
from data.load_datasets import carregar_dataset
from models.train_model import treinar_modelo
from explanations.pi_explanation import analisar_instancias, one_explanation

# Escolher dataset
datasets_disponiveis = ['iris', 'wine', 'breast_cancer', 'digits', 'banknote_authentication',
                        'wine_quality', 'heart_disease', 'parkinsons', 'car_evaluation', 'diabetes_binary']
print("Datasets disponíveis:")
for i, nome in enumerate(datasets_disponiveis):
    print(f"[{i}] {nome}")

dataset_idx = int(input("Digite o número do dataset desejado: "))
nome_dataset = datasets_disponiveis[dataset_idx]

# Carregaando dataset
X, y, class_names = carregar_dataset(nome_dataset)
print("Classes disponíveis:")
for i, nome_classe in enumerate(class_names):
    print(f"[{i}] {nome_classe}")

# Escolhendo classe 0
classe_0_idx = int(input("Digite o número da classe que será 0: "))
y_binario = [0 if label == classe_0_idx else 1 for label in y]

# Treinando modelo
modelo, X_test, y_test = treinar_modelo(X, y_binario, classe_0=0)

# Escolhendo instância para análise
idx_teste = int(input(f"Escolha um índice de 0 a {len(X_test)-1} para testar: "))
X_test_instancia = X_test.iloc[[idx_teste]].copy()

# Calculando PI-explicação usando a mesma lógica do main.py
Vs = X_test_instancia.iloc[0].to_dict()
feature_names = X_test.columns.tolist()

delta = []
w = modelo.coef_[0]
for i, feature in enumerate(feature_names):
    if w[i] < 0:
        delta.append((Vs[feature] - X[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - X[feature].min()) * w[i])

R = sum(delta) - modelo.decision_function(X_test_instancia)[0]

pi_explicacao = one_explanation(Vs, delta, R, feature_names, modelo, X_test_instancia, X)
print(f"Instância {idx_teste} - PI-Explicação: {pi_explicacao}")

# Criando uma cópia perturbada da instância removendo as features explicativas
X_test_perturbada = X_test_instancia.copy()
for feature_explicativa in pi_explicacao:
    feature = feature_explicativa.split(" - ")[0]  # Extraindo apenas o nome da feature
    X_test_perturbada[feature] = X[feature].mean()  # Substitui pelo valor médio da feature
    X_test_perturbada[feature] = X[feature].mean()  # Substitui pelo valor médio da feature

# Predição do modelo antes e depois da perturbação
classe_original = modelo.predict(X_test_instancia)[0]
classe_perturbada = modelo.predict(X_test_perturbada)[0]

print(f"Classe original: {classe_original}")
print(f"Classe após perturbação: {classe_perturbada}")

# Validação
if classe_original != classe_perturbada:
    print("A PI-explicação está coerente, pois a remoção das features alterou a predição.")
else:
    print("A PI-explicação pode estar incompleta ou imprecisa, pois a predição não mudou.")


Datasets disponíveis:
[0] iris
[1] wine
[2] breast_cancer
[3] digits
[4] banknote_authentication
[5] wine_quality
[6] heart_disease
[7] parkinsons
[8] car_evaluation
[9] diabetes_binary
Classes disponíveis:
[0] setosa
[1] versicolor
[2] virginica
Instância 28 - PI-Explicação: ['petal length (cm) - 1.4']
Classe original: 1
Classe após perturbação: 0
✔️ A PI-explicação está coerente, pois a remoção das features alterou a predição.


In [5]:
#ADICIONAR PERTURBAÇÃO NAS FEATURES DA INSTANCIA ESCOLHIDA PARA VALIDAR A PI-EXPLICAÇÃO
import numpy as np
import pandas as pd
from data.load_datasets import carregar_dataset
from models.train_model import treinar_modelo
from explanations.pi_explanation import analisar_instancias, one_explanation

# Escolher dataset
datasets_disponiveis = ['iris', 'wine', 'breast_cancer', 'digits', 'banknote_authentication',
                        'wine_quality', 'heart_disease', 'parkinsons', 'car_evaluation', 'diabetes_binary']
print("Datasets disponíveis:")
for i, nome in enumerate(datasets_disponiveis):
    print(f"[{i}] {nome}")

dataset_idx = int(input("Digite o número do dataset desejado: "))
nome_dataset = datasets_disponiveis[dataset_idx]

# Carregaando dataset
X, y, class_names = carregar_dataset(nome_dataset)
print("Classes disponíveis:")
for i, nome_classe in enumerate(class_names):
    print(f"[{i}] {nome_classe}")

# Escolhendo classe 0
classe_0_idx = int(input("Digite o número da classe que será 0: "))
y_binario = [0 if label == classe_0_idx else 1 for label in y]

# Treinando modelo
modelo, X_test, y_test = treinar_modelo(X, y_binario, classe_0=0)

# Escolhendo instância para análise
idx_teste = int(input(f"Escolha um índice de 0 a {len(X_test)-1} para testar: "))
X_test_instancia = X_test.iloc[[idx_teste]].copy()

# Calculando PI-explicação usando a mesma lógica do main.py
Vs = X_test_instancia.iloc[0].to_dict()
feature_names = X_test.columns.tolist()

delta = []
w = modelo.coef_[0]
for i, feature in enumerate(feature_names):
    if w[i] < 0:
        delta.append((Vs[feature] - X[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - X[feature].min()) * w[i])

R = sum(delta) - modelo.decision_function(X_test_instancia)[0]

pi_explicacao = one_explanation(Vs, delta, R, feature_names, modelo, X_test_instancia, X)
print(f"Instância {idx_teste} - PI-Explicação: {pi_explicacao}")

percentile_min = 5
percentile_max = 95
random_state=None

if random_state is not None:
    np.random.seed(random_state)

# Criando uma cópia perturbada da instância removendo as features explicativas
X_test_perturbada = X_test_instancia.copy()
for feature_explicativa in pi_explicacao:
    feature = feature_explicativa.split(" - ")[0]  # Extraindo apenas o nome da feature
    min_val = np.percentile(X[feature], percentile_min) # Calcula o percentil mínimo
    max_val = np.percentile(X[feature], percentile_max) # Calcula o percentil máximo

    # Gera um valor aleatório dentro do intervalo dos percentis
    X_test_perturbada[feature] = np.random.uniform(min_val, max_val)

# Predição do modelo antes e depois da perturbação
classe_original = modelo.predict(X_test_instancia)[0]
classe_perturbada = modelo.predict(X_test_perturbada)[0]

print(f"Classe original: {classe_original}")
print(f"Classe após perturbação: {classe_perturbada}")

# Validação
if classe_original != classe_perturbada:
    print("A PI-explicação está coerente, pois a remoção das features alterou a predição.")
else:
    print("A PI-explicação pode estar incompleta ou imprecisa, pois a predição não mudou.")


Datasets disponíveis:
[0] iris
[1] wine
[2] breast_cancer
[3] digits
[4] banknote_authentication
[5] wine_quality
[6] heart_disease
[7] parkinsons
[8] car_evaluation
[9] diabetes_binary
Classes disponíveis:
[0] setosa
[1] versicolor
[2] virginica
Instância 28 - PI-Explicação: ['petal length (cm) - 1.4']
Classe original: 1
Classe após perturbação: 0
A PI-explicação está coerente, pois a remoção das features alterou a predição.
